[Reference](https://anmol3015.medium.com/dont-just-create-plots-in-python-go-one-step-beyond-d60197ed6aa7)

In [1]:
# install the following libraries if not present already
! pip install dash 
! pip install dash-html-components 
! pip install dash-core-components 
! pip install plotly 
! pip install pytrends  #package for fetching google trends data 

     |████████████████████████████████| 8.5 MB 8.0 MB/s 
     |████████████████████████████████| 357 kB 49.7 MB/s 
  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=93b8980a7f541444b9629a33ae4db8e4601a5b03499bd8c47f3d208835fe25b8
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends


In [3]:
!pip install dash-bootstrap-components

     |████████████████████████████████| 209 kB 8.8 MB/s 


In [4]:
import dash
import dash_html_components as html   # for creating html components 
import dash_core_components as dcc   # for creating components such as graphs within the dash layout.
import plotly.graph_objects as go   # for creating plots 
import dash_bootstrap_components as dbc  # for creating layout in Dash
from pytrends.request import TrendReq  # for fetching google trends data 

In [5]:
# set up connection to google trends api by calling the TrendReq() function 
pytrend = TrendReq()

# Build Payload
pytrend.build_payload(kw_list=["Netflix"])

# Interest in Netflix over time 
trend_df = pytrend.interest_over_time().reset_index().tail(10)
trend_df = trend_df.rename(columns = {'Netflix':'Interest'})
trend_df.head()

# Interest in Netflix by region 
df = pytrend.interest_by_region()
int_by_reg = df['Netflix'].sort_values(ascending = False).head(10)

int_by_reg =int_by_reg.reset_index()
int_by_reg.columns = ['Country','Interest']
int_by_reg.head()

,Country,Interest
0,Argentina,100
1,Uruguay,100
2,Mexico,92
3,Colombia,90
4,Ireland,90


In [12]:
## STEP 1 : create the line chart within a function
def trend():
    fig = go.Figure([go.Scatter(x = trend_df['date'], y = trend_df['Interest'], marker_color = 'indianred'), 
             ])

    fig.update_layout(title = 'Interest in Netlfix over weeks',
                  title_x = 0.5,
              xaxis_title = 'Dates',
              yaxis_title = 'Interest Score', 
               barmode = 'group',
               xaxis=dict(showgrid=False),
          yaxis=dict(showgrid=False) ,
          plot_bgcolor='rgb(255,255,255)',
          yaxis_range=[0,100]
          )

    return fig

# bar chart showing the top countries where Netflix is searched  
def top_countries():
    fig = go.Figure([go.Bar(x = int_by_reg['Country'], y = int_by_reg['Score'], marker_color = 'indianred'),  #,name = 'Country'
                 ])

    fig.update_layout(title = 'Interest across regions for Netflix',
                      title_x = 0.5,
                  xaxis_title = 'Countries',
                  yaxis_title = 'Relative Interest Score', 
                   barmode = 'group',
                   xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False) ,
              plot_bgcolor='rgb(255,255,255)'
              )


    return fig 

In [13]:
## initialize the app and set the theme to Flatly. 
## Theme sets the color combinations and style of the elements such as data cards. 
app = dash.Dash(external_stylesheets = [ dbc.themes.FLATLY],)

## STEP 2 : Setting the body of the app where we are integrating the charts : 2 columns within one row 
body_app = dbc.Container([
    
     dbc.Row([
         
         # 1st column covering 6 columms for big size screens
         dbc.Col(
              dcc.Graph(id = 'trend', figure = trend()), # end of graph component
             style = {'height':'450px'},xs = 12, sm = 12, md = 6, lg = 6, xl = 6
                ),
         
         # 2nd column coving 6 columns for big size screens
          dbc.Col(
              dcc.Graph(id = 'regions', figure = top_countries()),
              style = {'height':'450px'},xs = 12, sm = 12, md = 6, lg = 6, xl = 6
              ),         
         ]), 
    
],fluid = True)  # Using Fluid = True, components occupy the empty space 

## STEP 3 : Set the layout of the app with the body of the dashboard
app.layout = html.Div(id = 'parent', children = [body_app])

if __name__ == "__main__":
    app.run_server()

In [9]:
# Step 1 : Creating the dataframe 
# ## Top 5 topics related to Netflix
df_related_topics = pytrend.related_topics()['Netflix']['rising'][:5]
df_related_topics.head()

# ## Top 10 queries related to Netflix
df_related_queries = pytrend.related_queries()['Netflix']['top'][:10]
df_related_queries.head()

,query,value
0,netflix movies,100
1,netflix series,78
2,movies on netflix,58
3,serie netflix,42
4,film netflix,41


In [11]:
# bar graph for showing top related topics 
def top_related_search():
    fig = go.Figure([go.Bar(x = df_related_topics['topic_title'], y = df_related_topics['value'], marker_color = 'indianred'),  #,name = 'Country'
              ])

    fig.update_layout(title = 'Topics related to Netflix',
                  title_x = 0.5,
              xaxis_title = 'Related Topics',
              yaxis_title = 'Score', 
                barmode = 'group',
                xaxis=dict(showgrid=False),
          yaxis=dict(showgrid=False) ,
          plot_bgcolor='rgb(255,255,255)'
          )

    return fig 


# bar graph for showing top related queries  
def top_related_queries():
    fig = go.Figure([go.Bar(x = df_related_queries['query'], y = df_related_queries['value'], marker_color = 'indianred'),  #,name = 'Country'
              ])

    fig.update_layout(title = 'Queries related for Netflix',
                  title_x = 0.5,
              xaxis_title = 'Related Queries',
              yaxis_title = 'Relative Score', 
                barmode = 'group',
                xaxis=dict(showgrid=False),
          yaxis=dict(showgrid=False) ,
          plot_bgcolor='rgb(255,255,255)'
          )

    return fig 

body_app = dbc.Container([
    
     dbc.Row([
         
         # 1st column covering 6 columms for big size screens
         dbc.Col(
              dcc.Graph(id = 'trend', figure = trend()
                        ),
             style = {'height':'450px'},xs = 12, sm = 12, md = 6, lg = 6, xl = 6),
         
         # 2nd column coving 6 columns for big size screens
          dbc.Col(
              dcc.Graph(id = 'regions', figure = top_countries()
                        ),
              style = {'height':'450px'},xs = 12, sm = 12, md = 6, lg = 6, xl = 6
              ),
         
         ]), 
     
     ### Adding the row with 2 columns having one graph component each  
      dbc.Row([
         
          dbc.Col(
              dcc.Graph(id = 'top_related', figure = top_related_search()
                        ),
              style = {'height':'450px'},xs = 12, sm = 12, md = 6, lg = 6, xl = 6
              ),

         
          dbc.Col(
              dcc.Graph(id = 'related_queries', figure = top_related_queries()),style = {'height':'450px'},xs = 12, sm = 12, md = 6, lg = 6, xl = 6
              ),
                  
          ])

    
],fluid = True)